In [3]:
import pandas as pd
import nltk
import re
import pickle
from sklearn.feature_extraction.text import TfidfVectorizer, ENGLISH_STOP_WORDS
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_colwidth', 100)
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\karan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [43]:
def tokenize_tweet(tweet):
    tokens = [word.lower() for sent in nltk.sent_tokenize(tweet) for word in nltk.word_tokenize(sent)]
    filtered_tokens = []
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            if token not in ['http','https','rt']:
                filtered_tokens.append(token)
    return filtered_tokens

In [2]:
df = pd.read_csv('iranian_tweets_csv_hashed.csv')

In [6]:
df.head()

,tweetid,userid,user_display_name,user_screen_name,user_reported_location,user_profile_description,user_profile_url,follower_count,following_count,account_creation_date,account_language,tweet_language,tweet_text,tweet_time,tweet_client_name,in_reply_to_tweetid,in_reply_to_userid,quoted_tweet_tweetid,is_retweet,retweet_userid,retweet_tweetid,latitude,longitude,quote_count,reply_count,like_count,retweet_count,hashtags,urls,user_mentions,poll_choices
0,533622371429543936,299148448,Maria Luis,marialuis91,"Nantes, France",journaliste indépendante/un vrai journaliste e...,NaN,8012,1450,2011-05-15,en,fr,@bellisarobz Ces photos illustrent parfaitemen...,2014-11-15 14:07,Twitter Web Client,5.336216e+17,574356455,NaN,False,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,[http://fr.awdnews.com/divertissements/5757-ce...,[574356455],NaN
1,527205814906654721,299148448,Maria Luis,marialuis91,"Nantes, France",journaliste indépendante/un vrai journaliste e...,NaN,8012,1450,2011-05-15,en,en,@ParkerLampe An inquiry by congress confirms t...,2014-10-28 21:10,Twitter Web Client,5.272054e+17,438542436,NaN,False,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,[http://www.awdnews.com/top-news/10080-an-inqu...,[438542436],NaN
2,545166827350134784,299148448,Maria Luis,marialuis91,"Nantes, France",journaliste indépendante/un vrai journaliste e...,NaN,8012,1450,2011-05-15,en,en,@hadeelhmaidi @wordpressdotcom CIA predict thi...,2014-12-17 10:41,Twitter Web Client,5.451665e+17,256625456,NaN,False,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,[http://www.awdnews.com/political/10404-cia-pr...,"[823905, 256625456]",NaN
3,538045437316321280,299148448,Maria Luis,marialuis91,"Nantes, France",journaliste indépendante/un vrai journaliste e...,NaN,8012,1450,2011-05-15,en,fr,@MartinYannis l'avis bizarre de marcel tonton ...,2014-11-27 19:03,Twitter Web Client,5.380451e+17,919513172,NaN,False,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,NaN,[http://fr.awdnews.com/divertissements/5796-to...,[919513172],NaN
4,530053681668841472,299148448,Maria Luis,marialuis91,"Nantes, France",journaliste indépendante/un vrai journaliste e...,NaN,8012,1450,2011-05-15,en,fr,@courrierinter Les laboratoires US de guerre b...,2014-11-05 17:47,Twitter Web Client,5.300444e+17,83864876,NaN,False,NaN,NaN,NaN,NaN,0.0,0.0,1.0,2.0,NaN,[http://fr.awdnews.com/soci%C3%A9t%C3%A9/5723-...,[83864876],NaN


In [7]:
columns = [col for col in df.columns if col not in ["userid","follower_count","tweet_language","tweet_text","tweet_time","like_count"]]
df.drop(columns=columns,inplace=True)

In [8]:
df.head()

,userid,follower_count,tweet_language,tweet_text,tweet_time,like_count
0,299148448,8012,fr,@bellisarobz Ces photos illustrent parfaitemen...,2014-11-15 14:07,0.0
1,299148448,8012,en,@ParkerLampe An inquiry by congress confirms t...,2014-10-28 21:10,0.0
2,299148448,8012,en,@hadeelhmaidi @wordpressdotcom CIA predict thi...,2014-12-17 10:41,0.0
3,299148448,8012,fr,@MartinYannis l'avis bizarre de marcel tonton ...,2014-11-27 19:03,0.0
4,299148448,8012,fr,@courrierinter Les laboratoires US de guerre b...,2014-11-05 17:47,1.0


In [9]:
df.shape

(1122936, 6)

In [10]:
df = df[df["tweet_language"]=="en"]

In [11]:
df.shape

(276946, 6)

In [12]:
df['tweet_time'] = pd.to_datetime(df['tweet_time'])

In [52]:
vectorizer = TfidfVectorizer(stop_words=ENGLISH_STOP_WORDS.union(["rt","http","https"]))
features = vectorizer.fit_transform(df["tweet_text"])

In [53]:
model = KMeans(n_clusters=4, init='k-means++', max_iter=200, n_init=5)
model.fit(features)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=200,
    n_clusters=4, n_init=5, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [ ]:
print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()
for i in range(4):
    print("Cluster %d:" % i),
    for ind in order_centroids[i, :10]:
        print(' %s' % terms[ind]),

In [68]:
df['cluster'] = model.labels_.tolist()

In [26]:
df2 = pd.read_csv('russian_201901_1_tweets_csv_hashed.csv')

c:\users\student\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [29]:
columns2 = [col for col in df2.columns if col not in ["userid","follower_count","account_language","tweet_text","tweet_time","like_count"]]
df2.drop(columns=columns2,inplace=True)

In [30]:
df2.head()

,userid,follower_count,account_language,tweet_text,tweet_time,like_count
0,2980186012,8685,en,RT @AmyMek: Meanwhile in New York City....\n\n...,2017-02-28 07:30,0.0
1,2980186012,8685,en,RT @bean3ie: Yazidis have suffered 74 genocide...,2017-01-29 09:03,0.0
2,2980186012,8685,en,RT @Johnatsrs1949: RETWEET THIS IF YOU ARE ON ...,2015-12-20 07:51,0.0
3,2980186012,8685,en,RT @Truthdig: New Emails Confirm @HillaryClint...,2016-06-05 00:03,0.0
4,2980186012,8685,en,RT @CathyTo47590555: #ElectionFinalThoughts #D...,2016-11-08 06:54,0.0


In [31]:
df2.shape

(927536, 6)

In [33]:
df2 = df2[df2["account_language"]=="en"]

In [34]:
df2.shape

(911267, 6)

In [55]:
df2['tweet_time'] = pd.to_datetime(df2['tweet_time'])

In [56]:
vectorizer2 = TfidfVectorizer(stop_words=ENGLISH_STOP_WORDS.union(["rt","http","https"]))
features2 = vectorizer2.fit_transform(df2["tweet_text"])

In [57]:
model2 = KMeans(n_clusters=3, init='k-means++', max_iter=200, n_init=5)
model2.fit(features2)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=200,
    n_clusters=3, n_init=5, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [58]:
print("Top terms per cluster:")
order_centroids2 = model2.cluster_centers_.argsort()[:, ::-1]
terms2 = vectorizer2.get_feature_names()
for i in range(3):
    print("Cluster %d:" % i),
    for ind in order_centroids2[i, :10]:
        print(' %s' % terms2[ind]),

Top terms per cluster:
Cluster 0:
 trump
 amp
 obama
 hillary
 cathyto47590555
 just
 people
 america
 clinton
 uthornsrawk
Cluster 1:
 islam
 lesson
 today
 saminfidel
 stopimportingislam
 islamistheproblem
 educateyourselfonislam
 bansharialaw
 infidel4life2
 stopmakingexcusesforislam
Cluster 2:
 realdonaldtrump
 maga
 americafirst
 trump
 qanon
 president
 makeamericagreatagain
 trumptrain
 potus
 great


In [63]:
df2['cluster'] = model2.labels_.tolist()

In [69]:
df.to_csv("iranian_tweets_cluster.csv")
df2.to_csv("russian_tweets_cluster.csv")
pickle.dump(model, open('model.sav', 'wb'))
pickle.dump(model2, open('model2.sav', 'wb'))
pickle.dump(vectorizer, open('vectorizer.sav', 'wb'))
pickle.dump(vectorizer2, open('vectorizer2.sav', 'wb'))

In [40]:
df2 = pd.read_csv('russian_tweets_cluster.csv')

c:\users\karan\appdata\local\programs\python\python36\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (0,1,3,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [41]:
df2.shape

(1189989, 8)

In [42]:
df2.dropna(subset=['cluster'],inplace=True)

In [43]:
df2['cluster'] = df2['cluster'].astype(int)

In [44]:
df2.shape

(911167, 8)

In [45]:
df2 = df2[(df2['cluster']==0)|(df2['cluster']==2)]

In [46]:
df2.shape

(880862, 8)

In [48]:
df2.to_csv('russian_tweets_app.csv')

In [4]:
df3 = pd.read_csv('ira_tweets_short.csv')

In [5]:
vectorizer3 = TfidfVectorizer(stop_words=ENGLISH_STOP_WORDS.union(["rt","http","https"]))
features3 = vectorizer3.fit_transform(df3["tweet_text"])

In [ ]:
model3 = KMeans(n_clusters=4, init='k-means++', max_iter=200, n_init=5)
model3.fit(features3)

In [7]:
import pickle
model3 = pickle.load(open("model3.sav", 'rb'))
vectorizer3 = pickle.load(open("vectorizer3.sav", 'rb'))
print("Top terms per cluster:")
order_centroids3 = model3.cluster_centers_.argsort()[:, ::-1]
terms3 = vectorizer3.get_feature_names()
for i in range(4):
    print("Cluster %d:" % i),
    for ind in order_centroids3[i, :10]:
        print(' %s' % terms3[ind])

Top terms per cluster:
Cluster 0:
 sports
 love
 don
 true
 know
 like
 rap
 want
 just
 people
Cluster 1:
 news
 world
 chicago
 local
 man
 police
 texas
 new
 business
 shooting
Cluster 2:
 politics
 just
 like
 people
 man
 new
 amp
 life
 obama
 time
Cluster 3:
 trump
 donald
 president
 politics
 just
 breaking
 obama
 maga
 video
 supporters


In [55]:
df3['cluster'] = model3.labels_.tolist()

In [18]:
order_centroids3 = order_centroids3[:,0:10]
dict3 = {}
for i in range(4):
    words = []
    for ind in order_centroids3[i]:
        words.append(terms3[ind])
    dict3[i] = words

In [61]:
df3.to_csv("ira_tweets_cluster.csv")
pickle.dump(model3, open('model3.sav', 'wb'))
pickle.dump(vectorizer3, open('vectorizer3.sav', 'wb'))
pickle.dump(order_centroids3, open('order_centroids3.sav', 'wb'))
pickle.dump(terms3, open('terms3.sav', 'wb'))

In [20]:
pickle.dump(dict3, open('dict3.sav', 'wb'))

In [23]:
dict3 = pickle.load(open(""+"dict3.sav", 'rb'))